In [1]:
import toml
import os
import paths

def load_config():
    config_path = os.path.join(paths.PF_DIR, "config.toml")
    return toml.load(config_path)

In [2]:
config = load_config()

In [3]:
from src.particle_filter.parameters import ModelParameters
from src.particle_filter.global_settings import GlobalSettings

global_settings = GlobalSettings(
        num_particles=config["filter_params"]["num_particles"],
        population=100000,
        location_code='04',
        final_date='2023-10-28',
        runtime=365,
        dt=config["filter_params"]["dt"],
        beta_prior=tuple(config["filter_params"]["beta_prior"]),
        seed_size=config["filter_params"]["seed_size"],
        ll_variance=50
    )

    # Initializing ModelParameters automatically grabs
    # params from the config.toml file.
config_path = os.path.join(paths.PF_DIR, "config.toml")
model_parameters = ModelParameters(config_path)

In [4]:
from src.particle_filter.particle_cloud import ParticleCloud
from src.particle_filter.transition import OUModel

particles = ParticleCloud(settings = global_settings,
                          transition = OUModel(config_path))

In [5]:
particles.states[1, :, 0]

Array([9.7346734e+04, 2.6532617e+03, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 1.9430254e-01], dtype=float32)

Nice. Initial state is as expected. [S, I, R, H, new_H, Beta]

In [6]:
for time in range(particles.settings.runtime - 1):
    particles.update_all_particles(time)

TypeError: unsupported operand type(s) for *: 'NoneType' and 'jaxlib.xla_extension.ArrayImpl'